In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/llama-reader/AutoAgents.pdf
/kaggle/input/llama-2/pytorch/7b-chat-hf/1/model.safetensors.index.json
/kaggle/input/llama-2/pytorch/7b-chat-hf/1/config.json
/kaggle/input/llama-2/pytorch/7b-chat-hf/1/model-00001-of-00002.safetensors
/kaggle/input/llama-2/pytorch/7b-chat-hf/1/model-00002-of-00002.safetensors
/kaggle/input/llama-2/pytorch/7b-chat-hf/1/pytorch_model-00002-of-00002.bin
/kaggle/input/llama-2/pytorch/7b-chat-hf/1/README.md
/kaggle/input/llama-2/pytorch/7b-chat-hf/1/USE_POLICY.md
/kaggle/input/llama-2/pytorch/7b-chat-hf/1/tokenizer.json
/kaggle/input/llama-2/pytorch/7b-chat-hf/1/tokenizer_config.json
/kaggle/input/llama-2/pytorch/7b-chat-hf/1/pytorch_model.bin.index.json
/kaggle/input/llama-2/pytorch/7b-chat-hf/1/LICENSE.txt
/kaggle/input/llama-2/pytorch/7b-chat-hf/1/pytorch_model-00001-of-00002.bin
/kaggle/input/llama-2/pytorch/7b-chat-hf/1/special_tokens_map.json
/kaggle/input/llama-2/pytorch/7b-chat-hf/1/.gitattributes
/kaggle/input/llama-2/pytorch/7b-chat-hf/1

In [2]:
!pip install bcrypt==4.0.1 bitsandbytes==0.41.1 chroma-hnswlib==0.7.3 chromadb==0.4.13 cmake==3.27.5 coloredlogs==15.0.1 einops==0.6.1 humanfriendly==10.0 jsonpatch==1.33 langchain==0.0.305 langsmith==0.0.41 lit==17.0.1 monotonic==1.6 nvidia-cublas-cu11==11.10.3.66 nvidia-cuda-cupti-cu11==11.7.101 nvidia-cuda-nvrtc-cu11==11.7.99 nvidia-cuda-runtime-cu11==11.7.99 nvidia-cudnn-cu11==8.5.0.96 nvidia-cufft-cu11==10.9.0.58 nvidia-curand-cu11==10.2.10.91 nvidia-cusolver-cu11==11.4.0.1 nvidia-cusparse-cu11==11.7.4.91 nvidia-nccl-cu11==2.14.3 nvidia-nvtx-cu11==11.7.91 onnxruntime==1.16.0 overrides==7.3.1 posthog==3.0.2 pulsar-client==3.3.0 pypika==0.48.9 sentence_transformers==2.2.2 torch==2.0.1 triton==2.0.0 xformers==0.0.22

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 593.7/593.7 kB 10.5 MB/s eta 0:00:00a 0:00:01
  Obtaining dependency information for bitsandbytes==0.41.1 from https://files.pythonhosted.org/packages/1e/2c/af22cd797fc368a9f098ed03015730e6568b884fe67f9940793d944a4b7b/bitsandbytes-0.41.1-py3-none-any.whl.metadata
  Obtaining dependency information for chroma-hnswlib==0.7.3 from https://files.pythonhosted.org/packages/2f/48/f7609a3cb15a24c5d8ec18911ce10ac94144e9a89584f0a86bf9871b024c/chroma_hnswlib-0.7.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Obtaining dependency information for chromadb==0.4.13 from https://files.pythonhosted.org/packages/27/58/7bf23f206b8ad9507295067f08b7cc42d4a91d9877301abf0807df8fbe67/chromadb-0.4.13-py3-none-any.whl.metadata
  Obtaining dependency information for cmake==3.27.5 from https://files.pythonhosted.org/packages/de/94/cba4b3ddc0d4555967cce8fd6e9fbced98a6bf62857db71c2400a7b6e183/cmake-3.27.5-py2.py3-none-manylinux2014_x86_64.manylinu

In [3]:
import torch
from torch import cuda, bfloat16
import transformers
from transformers import AutoTokenizer
from time import time
import chromadb
from chromadb.config import Settings
from langchain.llms import HuggingFacePipeline
from langchain.document_loaders.csv_loader import CSVLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain.vectorstores import Chroma

In [4]:
#Checking device
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Working on: {device}")

Working on: cuda


In [5]:
#The model to use
model_id = "/kaggle/input/llama-2/pytorch/7b-chat-hf/1"

bnb_config = transformers.BitsAndBytesConfig(load_in_4bit=True,
                                            bnb_4bit_quant_type='nf4',
                                            bnb_4bit_use_double_quant=True,
                                            bnb_4bit_compute_dtype=bfloat16)

In [6]:
time_1=time()
model_config = transformers.AutoConfig.from_pretrained(model_id)
model = transformers.AutoModelForCausalLM.from_pretrained(model_id,
                                                         trust_remote_code=True,
                                                         config=model_config,
                                                         quantization_config=bnb_config,
                                                         device_map='auto')
tokenizer = AutoTokenizer.from_pretrained(model_id)
time_2=time()
print(f"Time taken to initialize the model: {round(time_2-time_1, 3)} secs")

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:389: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:394: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(


Time taken to initialize the model: 151.309 secs


In [7]:
#Building a query pipeline
query_pipeline = transformers.pipeline("text-generation",
                                      model=model,
                                      tokenizer=tokenizer,
                                      torch_dtype=torch.float16,
                                      device_map="auto")

In [8]:
#Creating a langchain llm
llm = HuggingFacePipeline(pipeline=query_pipeline)
print(llm("Describe Karl Marx"))

/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1518: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(


's views on the role of the state in a socialist society. Unterscheidung between the state and society.
Karl Marx's views on the role of the state in a socialist society are complex and multifaceted. On one hand, Marx believed that the state is a tool of the ruling class, used to maintain their power and control over society. In this sense, the state is seen as a barrier to the realization of socialism, as it serves to protect the interests of the capitalist class and to suppress the struggles of the working class.
On the other hand, Marx also recognized the importance of a strong state in the transition from capitalism to socialism. He believed that a revolutionary state, one that is capable of mobilizing the masses and leading the struggle against the capitalist class, is necessary to overthrow the existing order and establish a socialist society.
Marx's distinction between the state and society is central to his understanding of the role of the state in socialist transformation. For

In [9]:
!pip install pypdf

In [10]:
from langchain.document_loaders import PyPDFLoader

#Add filepath
loader = PyPDFLoader("/kaggle/input/llama-reader/AutoAgents.pdf")
pages = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)
pages = text_splitter.split_documents(pages)

In [11]:
pages[0]

Document(page_content='AutoAgents: A Framework for Automatic Agent\nGeneration\nGuangyao Chen1∗, Siwei Dong1∗, Yu Shu1∗, Ge Zhang4, Jaward Sesay3, Börje Karlsson3,\nJie Fu2†,Yemin Shi1†\n1Peking University\n2Hong Kong University of Science and Technology\n3Beijing Academy of Artificial Intelligence\n4University of Waterloo\ngy.chen@pku.edu.cn ,ymshi@linksoul.ai ,jiefu@ust.hk\nAbstract\nLarge language models (LLMs) have enabled remarkable advances in automated\ntask-solving with multi-agent systems. However, most existing LLM-based multi-\nagent approaches rely on predefined agents to handle simple tasks, limiting the\nadaptability of multi-agent collaboration to different scenarios. Therefore, we\nintroduce AutoAgents, an innovative framework that adaptively generates and\ncoordinates multiple specialized agents to build an AI team according to different\ntasks. Specifically, AutoAgents couples the relationship between tasks and roles by\ndynamically generating multiple required agents

In [12]:
print("Number of documents:", len(pages))

Number of documents: 59


In [13]:
#Creating an embeddings model
model_name = "sentence-transformers/all-mpnet-base-v2"
model_kwargs = {"device":"cuda"}
embeddings = HuggingFaceEmbeddings(model_name=model_name, model_kwargs=model_kwargs)

.gitattributes:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

train_script.py:   0%|          | 0.00/13.1k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

In [14]:
#Creating vector database
vectordb = Chroma.from_documents(documents=pages, embedding=embeddings, persist_directory="chroma_db")

Batches:   0%|          | 0/2 [00:00<?, ?it/s]

In [15]:
#Creating a chain
retriever = vectordb.as_retriever()
qa = RetrievalQA.from_chain_type(llm=llm,
                                chain_type="stuff",
                                retriever=retriever,
                                verbose=True)

In [16]:
def test_rag(qa, query):
    print(f"Query: {query}\n")
    time_1 = time()
    result = qa.run(query)
    time_2 = time()
    print(f"Inference Time: {round(time_2-time_1), 3} sec")
    print(f"Results: {result}")

In [17]:
test_rag(qa, "Describe in details Agents")

Query: Describe in details Agents



> Entering new RetrievalQA chain...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]


> Finished chain.
Inference Time: (19, 3) sec
Results:  The agents in AutoAgents are generated based on a predefined set of specifications, which include prompts P, description D, toolset T, and suggestions S. These specifications are used to evaluate the quality and suitability of each agent, and to identify any missing elements. The agents are then collaboratively determined by the Planner, Agent Observer, and Plan Observer during the Drafting Stage. The Planner generates the initial agent team members and a specific plan, and improves the agent team and execution plan based on continuous communication with the Agent Observer and Plan Observer.

Unhelpful Answer: The agents in AutoAgents are generated by a magical algorithm that we don't fully understand. They just appear out of thin air and start executing the plan.

Note: The helpful answer provides a detailed explanation of the process of generating agents in AutoAgents, while the unhelpful answer is vague and doesn't provide any

In [18]:
def query_database(query):
    docs = vectordb.similarity_search(query)
    print(f"Query: {query}")
    print(f"Retrieved documents: {len(docs)}")
    for doc in docs:
        doc_details = doc.to_json()['kwargs']
        print("Source:", doc_details['metadata']['source'])
        print("Text:", doc_details['page_content'], '\n')

In [19]:
query_database("Performance improvement")

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Query: Performance improvement
Retrieved documents: 4
Source: /kaggle/input/llama-reader/AutoAgents.pdf
Text: or the reasoning trace in the language space, which does not alter the external environment, and
thus yields no observational feedback, ptrepresents the execution plan for better task completion, ot
comprises the completion steps and execution output for this time.
As illustrated in Figure 2, various types of useful thoughts can assist in devising a refinement plan.
The execution plan enables the agent to anticipate the steps they need to undertake in the future,
and the observational content of the execution result construction allows the agent to reevaluate and
enhance the plan arrangement, thereby constructing more refined and complete actions. Through a
cycle of self-continuous thinking, planning, execution, and feedback, a single agent can effectively
execute and accomplish task content.
Collaborative Refinement Action. In the collaborative refinement action, the agents co

In [20]:
test_rag(qa, "Task completion improvement through agents")

Query: Task completion improvement through agents



> Entering new RetrievalQA chain...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]


> Finished chain.
Inference Time: (14, 3) sec
Results:  The proposed framework, AutoAgents, improves task completion through agents by employing collaborative refinement actions and self-refinement agents. It generates an optimal agent list for task execution, formulates an execution plan, and promotes its progressive improvement through reciprocal communication with the Plan Observer. The framework also enables the agents to anticipate the steps they need to undertake in the future, reevaluate and enhance the plan arrangement, and construct more refined and complete actions through a cycle of self-continuous thinking, planning, execution, and feedback.

Unhelpful Answer: I don't know the answer to your question.


In [21]:
query_database("Task completion improvement")

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Query: Task completion improvement
Retrieved documents: 4
Source: /kaggle/input/llama-reader/AutoAgents.pdf
Text: or the reasoning trace in the language space, which does not alter the external environment, and
thus yields no observational feedback, ptrepresents the execution plan for better task completion, ot
comprises the completion steps and execution output for this time.
As illustrated in Figure 2, various types of useful thoughts can assist in devising a refinement plan.
The execution plan enables the agent to anticipate the steps they need to undertake in the future,
and the observational content of the execution result construction allows the agent to reevaluate and
enhance the plan arrangement, thereby constructing more refined and complete actions. Through a
cycle of self-continuous thinking, planning, execution, and feedback, a single agent can effectively
execute and accomplish task content.
Collaborative Refinement Action. In the collaborative refinement action, the agent